In [1]:
import os
import inspect
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling, rasp_to_graph
import numpy as np
from tracr.compiler.validating import validate
from typing import Union, TypeVar
from rasp_generator import map_primitives, sampling, utils
import networkx as nx

from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 


def compile_rasp_to_model(sop: rasp.SOp, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS"):
    return compiling.compile_rasp_to_model(
        sop,
        vocab=vocab,
        max_seq_len=max_seq_len,
        compiler_bos=compiler_bos
    )


TEST_INPUT = [1,2,3,4]

# Program Generation

In [10]:
sampler = sampling.ProgramSampler(validate_compilation=True)
errs = sampler.sample()
print(sampler.program([1,2,3,4]))
print()
print("Retries encountered:")
for err in errs:
    print(err)

ValueError: could not broadcast input array from shape (38,9) into shape (38,7)

In [11]:
# print program with and without test input
k = 1
utils.print_program(sampler.sops[-k])
print()
print()
utils.print_program(sampler.sops[-k], TEST_INPUT)

select_45 = Select(tokens, tokens, predicate=Comparison.EQ)
select_44 = Select(indices, indices, predicate=Comparison.EQ)
aggregate_43 = Aggregate(select_45, tokens)    # type: categorical
aggregate_42 = Aggregate(select_44, tokens)    # type: categorical
select_40 = Select(aggregate_42, aggregate_43, predicate=Comparison.EQ)
sequence_map_41 = SequenceMap(lambda x, y: x*y, aggregate_42, tokens)    # type: categorical
aggregate_39 = Aggregate(select_40, sequence_map_41)    # type: categorical


select_45 = Select(tokens, tokens, predicate=Comparison.EQ)
select_44 = Select(indices, indices, predicate=Comparison.EQ)
aggregate_43 = Aggregate(select_45, tokens)    # output: [1, 2, 3, 4]
aggregate_42 = Aggregate(select_44, tokens)    # output: [1, 2, 3, 4]
select_40 = Select(aggregate_42, aggregate_43, predicate=Comparison.EQ)
sequence_map_41 = SequenceMap(lambda x, y: x*y, aggregate_42, tokens)    # output: [1, 4, 9, 16]
aggregate_39 = Aggregate(select_40, sequence_map_41)    # output: [1, 

In [18]:
m = SequenceMap(lambda x, y: x*y, tokens, tokens)    # type: categorical
#m = Map(lambda x: x**2, tokens)
sel = Select(tokens, tokens, predicate=Comparison.EQ)
aggregate_39 = Aggregate(sel, m)    # type: categorical

program = aggregate_39
model = compiling.compile_rasp_to_model(program, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS")

ValueError: could not broadcast input array from shape (30,9) into shape (30,6)